# KcELECTRA: Korean comments ELECTRA

-깃헙 블로그 주소:  https://github.com/Beomi/KcELECTRA


- KcELECTRA는 위와 같은 특성의 데이터셋에 적용하기 위해, 온라인 뉴스에서 댓글과 대댓글을 수집해,  토크나이저와 ELECTRA모델을 처음부터 학습한 Pretrained ELECTRA 모델입니다.
- 기존 KcBERT 대비 데이터셋 증가 및 vocab 확장을 통해 상당한 수준으로 성능이 향상되었습니다.
- KcELECTRA는 Huggingface의 Transformers 라이브러리를 통해 간편히 불러와 사용할 수 있습니다. (별도의 파일 다운로드가 필요하지 않습니다.)

In [1]:
!pip install transformers

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split

In [3]:
#드라이브 연동
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# # Torch GPU 설정
# device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = torch.device(device_type)

# 데이터 전처리

In [5]:
data1 = pd.read_excel("/content/drive/MyDrive/Prog_All/Data/ko_단발성_대화_데이터셋.xlsx", engine='openpyxl')
data2 = pd.read_excel("/content/drive/MyDrive/Prog_All/Data/ko_연속적_대화_데이터셋.xlsx", engine='openpyxl')

In [6]:
data1 = data1[['Sentence','Emotion']]
data2 = data2[['Unnamed: 1','Unnamed: 2']]
data2.drop([0],axis=0,inplace=True)
data2.rename(columns={'Unnamed: 1':'Sentence','Unnamed: 2':'Emotion'},inplace=True)
data2.replace('ㅍ','공포',inplace=True)
data2.replace(['분','분ㄴ'],'분노',inplace=True)
data2.replace(['ㅈ중립','중림','ㄴ중립','줄'],'분노',inplace=True)


In [7]:
# nan 제거
data2 = data2.dropna(how='any')

In [8]:
data1['Emotion'].unique()
data2['Emotion'].unique()

array(['분노', '혐오', '중립', '놀람', '행복', '공포', '슬픔'], dtype=object)

In [9]:
data = pd.concat([data1,data2])
data.sample(n=10)

,Sentence,Emotion
20751,이것저것 여러 가지 하고 있습니다.,중립
36526,당분간 내가 집중공략 해야 하는 고객이 계시거든? 아침에 꼭 들렸다가 나와야하거든.,중립
1460,형이 무섭긴 무섭구나. 응?,분노
54725,"아이들도 쉽게 만들 수 있는 거, 말이지…… 그런데 너, 집안일은 다 잘했잖아?",중립
50466,그랬나. 서두르지 않으면 돌이킬 수 없게 될거야.,중립
11411,"애라이 저거 훔쳐가는 놈들 , 천벌 받는다",분노
16768,이래도 좌좀 좌빨 소리 지껄이는건 참 대단,분노
30849,아뇨. 원랜 되게 좋아하는데.. 오늘은 컨디션이 좀..,중립
35373,"저번 모임에도 안 나오시고, 한달에 반은 인도에 있다면서요?",중립
7794,남녀평등 수준이 한국보다 높은 나라들이 저 정도,놀람


In [10]:
data.drop_duplicates(['Sentence','Emotion'],inplace=True) # 중복행 제거
len(data)

90116

In [11]:
num_labeling_dics ={
    '공포': 0,
    '놀람': 1,
    '분노': 2,
    '슬픔': 3,
    '중립': 4,
    '행복': 5,
    '혐오': 6
  }

In [12]:
# 감정 -> 숫자 레이블링
for label_class in num_labeling_dics:
    data.loc[(data['Emotion'] == label_class), 'Emotion'] = num_labeling_dics[label_class]

In [13]:
# data_list = []
# for q, label in zip(data['Sentence'], data['Emotion'])  :
#     check_data = []
#     check_data.append(q)
#     check_data.append(str(label))
#     data_list.append(check_data)

In [14]:
texts = []
labels = []
for q, label in zip(data['Sentence'], data['Emotion'])  :
    texts.append(q)
    labels.append((label))

In [ ]:
texts

In [ ]:
labels

In [17]:
# print(data_list[0])
# print(data_list[6000])
# print(data_list[12000])
# print(data_list[18000])
# print(data_list[24000])
# print(data_list[30000])
# print(data_list[-1])

In [18]:
data['Emotion'].value_counts()

4    45641
1     9866
2     9238
3     7167
5     7015
6     5621
0     5568
Name: Emotion, dtype: int64

# WandB

In [19]:

!pip install wandb

In [20]:

!wandb login

wandb: Currently logged in as: tracy110410 (team_5g). Use `wandb login --relogin` to force relogin


In [21]:
import wandb
wandb.init(project="CJ_kcBERT", entity='tracy110410')


wandb: Currently logged in as: tracy110410. Use `wandb login --relogin` to force relogin


# Custom DataSet 구성

In [22]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'label': label}


In [23]:
def check_label(label):
  if label['공포'] == 0:
    return 0
  elif label['놀람'] == 1:
    return 1
  elif label['분노'] == 2:
    return 2
  elif label['슬픔'] == 3:
    return 3
  elif label['중립'] == 4:
    return 4
  elif label['행복'] == 5:
    return 5
  elif label['혐오'] == 6:
    return 6

In [24]:
# KcELECTRA 모델과 토크나이저 로드
model_name = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7)


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
# Setting parameters
max_length = 128
learning_rate = 1e-5
num_epochs = 5
batch_size = 64
random_seed=123

In [26]:
labels = torch.tensor(labels, dtype=torch.long)
dataset = CustomDataset(texts, labels, tokenizer, max_length)

In [27]:
# config
wandb.config ={
  "learning_rate": learning_rate,
  "epochs": num_epochs,
  "batch_size": batch_size,
  "seed": random_seed
}

In [28]:
# Train / Test set 분리
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

In [29]:
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:
train_data

In [31]:
data_train = CustomDataset(train_data, labels, tokenizer, max_length)
data_test = CustomDataset(test_data, labels, tokenizer, max_length)

In [32]:

# 옵티마이저 및 손실 함수 설정
optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [33]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

In [34]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

model = model.to(device)

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


https://breezymind.com/kcbert-find-tuning/

# 모델 재학습과 평가

In [35]:

# 모델 재학습
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # 그래디언트 초기화
        optimizer.zero_grad()
        # 모델에 입력을 주어 예측을 생성합니다.
        outputs = model(input_ids, attention_mask=attention_mask)
        # 모델 출력에서 로짓(분류에 대한 점수)을 얻습니다.
        logits = outputs.logits
        # 손실을 계산합니다.
        loss = criterion(logits, labels)
        # 역전파를 통해 그래디언트 계산
        loss.backward()
        # 옵티마이저를 사용해 가중치를 업데이트
        optimizer.step()
        # 에포크 전체 손실을 누적합니다.
        total_loss += loss.item()

    # 에포크 평균 손실 계산
    avg_loss = total_loss / len(train_dataloader)
    # 에포크별 손실 출력
    print(f"Epoch {epoch+1}/{num_epochs} - Avg Loss: {avg_loss:.4f}")

    # 모델 평가
    model.eval()
    val_total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for val_batch in valid_dataloader:
            # Validation 데이터 가져오기
            val_input_ids = val_batch['input_ids']
            val_attention_mask = val_batch['attention_mask']
            val_labels = val_batch['label']

            val_input_ids = val_input_ids.to(device)
            val_attention_mask = val_attention_mask.to(device)
            val_labels = val_labels.to(device)

            # 모델 예측
            val_outputs = model(val_input_ids, attention_mask=val_attention_mask)
            val_logits = val_outputs.logits

            # 손실 계산
            val_loss = criterion(val_logits, val_labels)
            val_total_loss += val_loss.item()

            # 정확도 계산
            val_preds = val_logits.argmax(dim=1)
            correct += (val_preds == val_labels).sum().item()
            total += val_labels.size(0)

    val_avg_loss = val_total_loss / len(valid_dataloader)
    val_accuracy = correct / total
    print(f"Validation Loss: {val_avg_loss:.4f} - Validation Accuracy: {val_accuracy:.4f}")

Epoch 1/5 - Avg Loss: 1.0225
Validation Loss: 0.8280 - Validation Accuracy: 0.7022
Epoch 2/5 - Avg Loss: 0.7718
Validation Loss: 0.8002 - Validation Accuracy: 0.7118
Epoch 3/5 - Avg Loss: 0.6907
Validation Loss: 0.8055 - Validation Accuracy: 0.7116
Epoch 4/5 - Avg Loss: 0.6152
Validation Loss: 0.8356 - Validation Accuracy: 0.7074
Epoch 5/5 - Avg Loss: 0.5743
Validation Loss: 0.8774 - Validation Accuracy: 0.7052


# 모델 저장 및 로드

In [ ]:
# 모델 저장
model_save_path = "kc_bert_emotion_classifier.pth"
torch.save(model.state_dict(), model_save_path)

# 모델 아키텍처 생성
loaded_model = AutoModelForSequenceClassification.from_pretrained("beomi/kcbert-base", num_labels=11)

# 저장된 가중치 불러오기
loaded_model.load_state_dict(torch.load(model_save_path))

# 모델을 평가 모드로 설정
loaded_model.eval()

#  모델 테스트 하기

In [38]:
def valid_label(label):
    if label == 0:
        return '공포'
    elif label == 1:
        return '놀람'
    elif label == 2:
        return '분노'
    elif label == 3:
        return '슬픔'
    elif label == 4:
        return '중립'
    elif label == 5:
        return '행복'
    elif label == 6:
        return '혐오'

In [45]:
# 입력 데이터 준비
input_data = [
    "만 개의 요리 레시피 감사합니다.",
    "백종원은 우리집 요리사.",
    "은근슬쩍 나올려고하네....?",
    "한심하다 한심한 인간들 득실득실해 ㅋㅋ",
    "가을이다...벌써 가을....",
    "아이폰이 답이다.......",
    "와...사진으로만으로도 압도되는데?.....",
    "내부자들 현실판 미쳤다 현실세계 ㄷㄷㄷㄷㄷ",
    "집에 가면 녹초가 된다고 너무 피곤해 요즘.....",
    "밥 먹을 힘도 없다 ㅠㅠ..",
    "너가 왠일이여???",
    "배고파서 짜증나.....",
    "아몬드브리즈 정말 맛있군"
    ]

In [46]:
import torch

# Set the device (CPU or GPU) based on the availability of GPUs
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Move the model to the selected device
model.to(device)

# Tokenize and encode your input_data on the same device
input_encodings = tokenizer(input_data, padding=True, truncation=True, return_tensors="pt").to(device)

# Ensure the model is in evaluation mode
model.eval()

# Pass the input data to the model
with torch.no_grad():
    output = model(**input_encodings)

# The rest of your code for prediction remains the same
logits = output.logits
predicted_labels = logits.argmax(dim=1)

for i, input_text in enumerate(input_data):
    predicted_label = predicted_labels[i].item()
    print(f"Input: {input_text} - Predicted Label: {valid_label(predicted_label)}")


Input: 만 개의 요리 레시피 감사합니다. - Predicted Label: 행복
Input: 백종원은 우리집 요리사. - Predicted Label: 중립
Input: 은근슬쩍 나올려고하네....? - Predicted Label: 놀람
Input: 한심하다 한심한 인간들 득실득실해 ㅋㅋ - Predicted Label: 혐오
Input: 가을이다...벌써 가을.... - Predicted Label: 중립
Input: 아이폰이 답이다....... - Predicted Label: 중립
Input: 와...사진으로만으로도 압도되는데?..... - Predicted Label: 놀람
Input: 내부자들 현실판 미쳤다 현실세계 ㄷㄷㄷㄷㄷ - Predicted Label: 놀람
Input: 집에 가면 녹초가 된다고 너무 피곤해 요즘..... - Predicted Label: 슬픔
Input: 밥 먹을 힘도 없다 ㅠㅠ.. - Predicted Label: 슬픔
Input: 너가 왠일이여??? - Predicted Label: 놀람
Input: 배고파서 짜증나..... - Predicted Label: 슬픔
Input: 아몬드브리즈 정말 맛있군 - Predicted Label: 행복


In [50]:
def predict(sentence, model, tokenizer):
    # Move the model to the device
    device = next(model.parameters()).device

    input_sentence = [sentence]
    input_encodings = tokenizer(input_sentence, padding=True, truncation=True, return_tensors="pt").to(device)

    # 모델에 입력 데이터 전달
    with torch.no_grad():
        output = model(**input_encodings)

    # 예측 결과 확인
    logits = output.logits
    predicted_labels = logits.argmax(dim=1)

    # 예측 결과 출력
    for i, input_text in enumerate(input_sentence):
        predicted_label = predicted_labels[i].item()
        print(f"Input: {input_text} - Predicted Label: {valid_label(predicted_label)}")


In [51]:

#질문 무한반복하기! 0 입력시 종료
while True:
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        print("감정 분석을 종료합니다.")
        break
    predict(sentence, model, tokenizer)
    print("\n")

하고싶은 말을 입력해주세요 : 아몬드 브리즈는 너무 맛있지만 약간 비싸서
Input: 아몬드 브리즈는 너무 맛있지만 약간 비싸서 - Predicted Label: 행복


하고싶은 말을 입력해주세요 : 반갑습니다. 오늘 아침에는 뭐 드셨어요?
Input: 반갑습니다. 오늘 아침에는 뭐 드셨어요? - Predicted Label: 중립


하고싶은 말을 입력해주세요 : 지금 잠도 못자고 모델 돌리고 있는 나의 모습.....너무 피곤하고 자고 싶어!!!!ㅜㅜㅜㅜ
Input: 지금 잠도 못자고 모델 돌리고 있는 나의 모습.....너무 피곤하고 자고 싶어!!!!ㅜㅜㅜㅜ - Predicted Label: 슬픔


하고싶은 말을 입력해주세요 : 어쩌라고 세상아
Input: 어쩌라고 세상아 - Predicted Label: 분노


하고싶은 말을 입력해주세요 : 0
감정 분석을 종료합니다.
